In [2]:
INF = 1e6
X, O, BLANK = 'X', 'O', ' '
UTILITY = {X: INF, O: -INF, None: 0}
N = 6
DOWN, UP = 1, -1

class State():
    
    def __init__(self, turn, state=None):
        self.board = X*N + BLANK*(N-2)*N + O*N
        
        if state is not None:
            self.board = state.board
        
        self.turn = turn
            
    def utility(self):
        ''' Returns the utility of a particular state '''
        # If terminal state, score accordingly
        if self.terminal_test() is not None:
            if self.terminal_test() == X:
                return INF
            return -INF
        
        # Else return score according to piece count ratio
        return self.board.count(X)/(self.board.count(O)+1)   
            
    def move(self, move):
        ''' Moves players piece from Start to End index '''
        s, e = move
        player = self.board[s]
        # Remove piece
        self.board = self.board[:s] + BLANK + self.board[s+1:]
        # Place piece
        self.board = self.board[:e] + player + self.board[e+1:] 
        self.turn = X if self.turn == O else O
    
    def terminal_test(self):
        ''' Returns the victorious player if there is one '''
        # All pieces taken
        if X not in self.board:
            return O
        if O not in self.board:
            return X
        
        # Check if either player has crossed board
        if X in self.board[N*(N-1):]:
            return X
        if O in self.board[:N]:
            return O
        
        # Check if player cannot move, (return opponent as winner if so)
        if len(self.actions()) == 0:
            return X if self.turn == O else O
        
        return None
            
    def __str__(self):        
        return (2*N+1)*'_' + ''.join(['\n|' + ''.join([e + '|' for e in self.board[i:i+N]]) + ' ' + \
                                     ''.join([str(j).rjust(3) for j in range(i, i+N)]) for i in range(0, N**2, N)])    
    
    def __repr__(self):
        return str(self)
    
    def actions(self):
        ''' Returns a list of possible actions for the player '''
        player   = X if self.turn == X else O
        opponent = O if self.turn == X else X
        direction = DOWN if player == X else UP
        
        # Find pieces
        lst = [i for i in range(N**2) if self.board[i] == player]

        actions = []
        for i in lst:
            # Check vertical is valid and free
            if (0 <= i+N*direction < N**2) and (self.board[i+N*direction] == BLANK):
                actions.append((i, i+N*direction))

            # Check diagonals are valid and taken by opposition
            # If not on left, check left
            new_i = i + N*direction - 1
            if (i%N != 0) and (0 <= new_i) and (self.board[new_i] == opponent):
                actions.append((i, new_i))

            # If not on right, check right
            new_i = i + N*direction + 1
            if ((i+1)%N != 0) and (new_i < N**2) and (self.board[new_i] == opponent):
                actions.append((i, new_i))
                    
        return actions
    
def ab_pruning(state, depth, α, β, isMax):
    if (depth == 0) or (state.terminal_test() is not None):
        return state.utility()
    
    if isMax:
        maxV = -INF
        for a in state.actions():
            result = State(state.turn, state)
            result.move(a)
            value = ab_pruning(result, depth-1, α, β, isMax=False)
            maxV = max(maxV, value)
            α = max(α, value)
            if β <= α:
                break
        return maxV
    else:
        minV = +INF
        for a in state.actions():
            result = State(state.turn, state)
            result.move(a)
            value = ab_pruning(result, depth-1, α, β, isMax=True)
            minV = min(minV, value)
            β = min(β, value)
            if β <= α:
                break
        return minV
    
def best_move2(state, depth):
    lst = []
    for a in state.actions():
        result = State(state.turn, state)
        result.move(a)
        lst.append((ab_pruning(result, depth, -INF, INF, isMax=False), a))
    
    return max(lst)[1] 

s = State(O)
s

_____________
|X|X|X|X|X|X|   0  1  2  3  4  5
| | | | | | |   6  7  8  9 10 11
| | | | | | |  12 13 14 15 16 17
| | | | | | |  18 19 20 21 22 23
| | | | | | |  24 25 26 27 28 29
|O|O|O|O|O|O|  30 31 32 33 34 35

In [4]:
DEPTH = 6

def game(ai_first=False):
    turn = X if ai_first else O
    s = State(turn)
    print(s)
    
    isMax = True if ai_first else False
    
    while s.terminal_test() is None:
        if not isMax:
            print('\n', list(enumerate(s.actions())), sep='')
            i = int(input())
            s.move(s.actions()[i])
        else:
            s.move(best_move2(s, DEPTH))
        
        print(s)
        isMax = not isMax
    print('\nGame Over!')

game(ai_first=False)

_____________
|X|X|X|X|X|X|   0  1  2  3  4  5
| | | | | | |   6  7  8  9 10 11
| | | | | | |  12 13 14 15 16 17
| | | | | | |  18 19 20 21 22 23
| | | | | | |  24 25 26 27 28 29
|O|O|O|O|O|O|  30 31 32 33 34 35

[(0, (30, 24)), (1, (31, 25)), (2, (32, 26)), (3, (33, 27)), (4, (34, 28)), (5, (35, 29))]
0
_____________
|X|X|X|X|X|X|   0  1  2  3  4  5
| | | | | | |   6  7  8  9 10 11
| | | | | | |  12 13 14 15 16 17
| | | | | | |  18 19 20 21 22 23
|O| | | | | |  24 25 26 27 28 29
| |O|O|O|O|O|  30 31 32 33 34 35
_____________
|X|X|X|X|X| |   0  1  2  3  4  5
| | | | | |X|   6  7  8  9 10 11
| | | | | | |  12 13 14 15 16 17
| | | | | | |  18 19 20 21 22 23
|O| | | | | |  24 25 26 27 28 29
| |O|O|O|O|O|  30 31 32 33 34 35

[(0, (24, 18)), (1, (31, 25)), (2, (32, 26)), (3, (33, 27)), (4, (34, 28)), (5, (35, 29))]
1
_____________
|X|X|X|X|X| |   0  1  2  3  4  5
| | | | | |X|   6  7  8  9 10 11
| | | | | | |  12 13 14 15 16 17
| | | | | | |  18 19 20 21 22 23
|O|O| | | | |  24 25 26 27 28 

In [114]:
def negaScout(state, a, b, depth):
    if (depth == 0) or state.utility is not None:
        return state.utility()
    best = -INF
    n = b
    for action in state.actions():
        result = State(state.turn, state)
        result.move(action)
        v = -negaScout(result, -n, -a, depth-1)
        if v > best:
            if (n == b) or (depth <= 2):
                best = v
            else:
                best = -negaScout(result, -b, -v, depth-1)
                
        if best > a:
            a = best
        if a >= b:
            return a
        n = a + 1
    return best

def best_move3(state, depth):
    lst = []
    for a in state.actions():
        result = State(state.turn, state)
        result.move(a)
        lst.append((negaScout(result, -INF, INF, depth), a))
    
    return max(lst)[1] 

DEPTH = 8

def game(ai_first=False):
    turn = X if ai_first else O
    s = State(turn)
    print(s)
    
    isMax = True if ai_first else False
    
    while s.terminal_test() is None:
        if not isMax:
            print('\n', list(enumerate(s.actions())), sep='')
            i = int(input())
            s.move(s.actions()[i])
        else:
            s.move(best_move3(s, DEPTH))
        
        print(s)
        isMax = not isMax
    print('\nGame Over!')

game(ai_first=False)

_____________
|X|X|X|X|X|X|   0  1  2  3  4  5
| | | | | | |   6  7  8  9 10 11
| | | | | | |  12 13 14 15 16 17
| | | | | | |  18 19 20 21 22 23
| | | | | | |  24 25 26 27 28 29
|O|O|O|O|O|O|  30 31 32 33 34 35

[(0, (30, 24)), (1, (31, 25)), (2, (32, 26)), (3, (33, 27)), (4, (34, 28)), (5, (35, 29))]
0
_____________
|X|X|X|X|X|X|   0  1  2  3  4  5
| | | | | | |   6  7  8  9 10 11
| | | | | | |  12 13 14 15 16 17
| | | | | | |  18 19 20 21 22 23
|O| | | | | |  24 25 26 27 28 29
| |O|O|O|O|O|  30 31 32 33 34 35
_____________
|X|X|X|X|X| |   0  1  2  3  4  5
| | | | | |X|   6  7  8  9 10 11
| | | | | | |  12 13 14 15 16 17
| | | | | | |  18 19 20 21 22 23
|O| | | | | |  24 25 26 27 28 29
| |O|O|O|O|O|  30 31 32 33 34 35

[(0, (24, 18)), (1, (31, 25)), (2, (32, 26)), (3, (33, 27)), (4, (34, 28)), (5, (35, 29))]
0
_____________
|X|X|X|X|X| |   0  1  2  3  4  5
| | | | | |X|   6  7  8  9 10 11
| | | | | | |  12 13 14 15 16 17
|O| | | | | |  18 19 20 21 22 23
| | | | | | |  24 25 26 27 28 

In [108]:
def best_move3(state, depth):
    lst = []
    for a in state.actions():
        result = State(state.turn, state)
        result.move(a)
        lst.append((negaScout(result, -INF, INF, depth), a))
    
    return max(lst)[1] 

DEPTH = 8

def game(ai_first=False):
    turn = X if ai_first else O
    s = State(turn)
    print(s)
    
    isMax = True if ai_first else False
    
    while s.terminal_test() is None:
        if not isMax:
            print('\n', list(enumerate(s.actions())), sep='')
            i = int(input())
            s.move(s.actions()[i])
        else:
            s.move(best_move2(s, DEPTH))
        
        print(s)
        isMax = not isMax
    print('\nGame Over!')

game(ai_first=False)

(_______
 |X|X|X|   0  1  2
 | | | |   3  4  5
 |O|O|O|   6  7  8, 'O')

_____________
|X|X|X|X|X|X|   0  1  2  3  4  5
| | | | | | |   6  7  8  9 10 11
| | | | | | |  12 13 14 15 16 17
| | | | | | |  18 19 20 21 22 23
| | | | | | |  24 25 26 27 28 29
|O|O|O|O|O|O|  30 31 32 33 34 35

[(0, (30, 24)), (1, (31, 25)), (2, (32, 26)), (3, (33, 27)), (4, (34, 28)), (5, (35, 29))]
1
_____________
|X|X|X|X|X|X|   0  1  2  3  4  5
| | | | | | |   6  7  8  9 10 11
| | | | | | |  12 13 14 15 16 17
| | | | | | |  18 19 20 21 22 23
| |O| | | | |  24 25 26 27 28 29
|O| |O|O|O|O|  30 31 32 33 34 35
_____________
|X|X|X|X|X| |   0  1  2  3  4  5
| | | | | |X|   6  7  8  9 10 11
| | | | | | |  12 13 14 15 16 17
| | | | | | |  18 19 20 21 22 23
| |O| | | | |  24 25 26 27 28 29
|O| |O|O|O|O|  30 31 32 33 34 35

[(0, (25, 19)), (1, (30, 24)), (2, (32, 26)), (3, (33, 27)), (4, (34, 28)), (5, (35, 29))]
4
_____________
|X|X|X|X|X| |   0  1  2  3  4  5
| | | | | |X|   6  7  8  9 10 11
| | | | | | |  12 13 14 15 16 17
| | | | | | |  18 19 20 21 22 23
| |O| | |O| |  24 25 26 27 28 

In [76]:
def best_move(state, depth):
    lst = []
    for a in state.actions(isMax=True):
        result = State(state)
        result.move(a)
        lst.append((minimax(result, depth, isMax=False), a))
    
    return max(lst)[1]    

def minimax(state, depth, isMax):
    if (depth == 0) or (state.terminal_test(isMax) is not None):
        return state.utility(isMax)
    
    if isMax:
        maxEval = -INF
        for a in state.actions(isMax):
            result = State(state)
            result.move(a)
            maxEval = max(maxEval, minimax(result, depth-1, isMax=False))
        return maxEval
    else:
        minEval = +INF
        for a in state.actions(isMax):
            result = State(state)
            result.move(a)
            minEval = min(minEval, minimax(result, depth-1, isMax=True))
        return minEval

In [77]:
def mtdf(root, guess, depth):
    g = guess
    upperBound = +INF
    lowerBound = -INF
    while True:
        if g == lowerBound:
            β = g+1
        else:
            β = g
        g = ab_memory(root, β-1, β, depth)
        if g < β:
            upperBound = g
        else:
            lowerBound = g
            
        if lowerBound >= upperBound:
            return g
        
table = {}
def ab_memory(root, a, b, depth):
    if str(root) in table:
        v = table[str(root)]
        if v[0] >= b:
            return v[0]
        if v[1] <= a:
            return v[1]
        a = max(a, v[0])
        b = min(b, v[1])
    
    if d == 0:
        g = root.utility()
    elif root.utility() == INF:
        g = -INF
        A = a
        